## Library imports

In [1]:
# basic imports
import os
import gc
import math
import glob
import random
import itertools
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# DL library imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from  torch.cuda.amp import autocast, GradScaler

# metrics calculation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold

# basic plotting library
import matplotlib.pyplot as plt

# interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

import warnings  
warnings.filterwarnings('ignore')

## Config parameters

In [2]:
class CFG:
    # pipeline parameters
    SEED        = 42
    TRAIN       = True
    LR_FIND     = False
    TEST        = False
    N_FOLDS     = 5 
    N_EPOCHS    = 50
    TEST_BATCH_SIZE  = 32
    TRAIN_BATCH_SIZE = 16
    NUM_WORKERS      = 4
    DATA_FRAC        = 1.0
    FOLD_TO_TRAIN    = [0, 1, 2, 3, 4] # 

    # model parameters
    MODEL_ARCH  = 'MLP'
    MODEL_NAME  = 'mlp_v3'
    WGT_PATH    = ''
    WGT_MODEL   = ''
    PRINT_N_EPOCH = 2
    
    # scheduler variables
    MAX_LR    = 1e-2
    MIN_LR    = 1e-4
    SCHEDULER = 'CosineAnnealingWarmRestarts'  # ['ReduceLROnPlateau', 'None', OneCycleLR', ','CosineAnnealingLR']
    T_0       = 10     # CosineAnnealingWarmRestarts
    T_MULT    = 2      # CosineAnnealingWarmRestarts
    T_MAX     = 2.5    # CosineAnnealingLR

    # optimizer variables
    OPTIMIZER     = 'Adam'
    WEIGHT_DECAY  = 1e-6
    GRD_ACC_STEPS = 1
    MAX_GRD_NORM  = 1000

In [3]:
floor_map = {"B2": -2, "B1": -1, "F1": 0, "F2": 1, "F3": 2, "F4": 3, "F5": 4, "F6": 5, "F7": 6, "F8": 7, "F9": 8,
             "1F": 0, "2F": 1, "3F": 2, "4F": 3, "5F": 4, "6F": 5, "7F": 6, "8F": 7, "9F": 8}

minCount = 1
rssiFillerValue = -999.0
dtFillerValue   = 1000.0
freqFillerValue = 0
outputDir = 'referencePublicNotebooks/wiFiFeatures'
modelOutputDir = 'modelSaveDir'
sampleCsvPath = 'sample_submission.csv'

buildingsList = glob.glob(f"{outputDir}/*.csv")
print([x.split('/')[-1] for x in buildingsList])

['5a0546857ecc773753327266_train.csv']


## Helper functions

In [4]:
def getBuildingName(buildingCsvPath):
    fileName = buildingCsvPath.split('/')[-1]
    buildingName = fileName.rstrip('_train.csv')
    return buildingName

In [5]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_trainable_params

In [6]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(CFG.SEED)

In [7]:
def competitionMetric(preds, targets):
    """ The metric used in this competition """
    # position error
    meanPosPredictionError = torch.mean(torch.sqrt(
                             torch.square(torch.subtract(preds[:,0], targets[:,0])) + 
                             torch.square(torch.subtract(preds[:,1], targets[:,1]))))
    # error in floor prediction
    meanFloorPredictionError = torch.mean(15 * torch.abs(preds[:,2] - targets[:,2]))
    return meanPosPredictionError, meanFloorPredictionError

In [8]:
def getOptimizer(model : nn.Module):    
    if CFG.OPTIMIZER == 'Adam':
        optimizer = optim.Adam(model.parameters(), weight_decay=CFG.WEIGHT_DECAY, lr=CFG.MAX_LR)
    else:
        optimizer = optim.SGD(model.parameters(), weight_decay=CFG.WEIGHT_DECAY, lr=CFG.MAX_LR, momentum=0.9)
    return optimizer

In [9]:
def getScheduler(optimizer, dataloader_train):
    if CFG.SCHEDULER == 'OneCycleLR':
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr= CFG.MAX_LR, epochs = CFG.N_EPOCHS, 
                          steps_per_epoch = len(dataloader_train), pct_start=0.25, div_factor=10, anneal_strategy='cos')
    elif CFG.SCHEDULER == 'CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=CFG.T_MULT, eta_min=CFG.MIN_LR, last_epoch=-1)
    elif CFG.SCHEDULER == 'CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_MAX * len(dataloader_train), eta_min=CFG.MIN_LR, last_epoch=-1)
    else:
        scheduler = None
    return scheduler

In [10]:
def getBuildingData(buildingCsvPath):
    # read building data 
    data = pd.read_csv(buildingCsvPath)
    
    # use fraction if needed
    if CFG.DATA_FRAC < 1:
        data = data.sample(frac=CFG.DATA_FRAC).reset_index(drop=True)

    """
    Incase freq signal is not needed
    
    # There are 4 columns for y values in csv, reamining are features
    # total features = 3 * [rssi, dt, freq]
    # hence unique wifi ids = totalFeatures / 3
    # numWiFiIds = int((data.shape[1] - 4) / 3)
    
    # separate into features and target variables
    X = data.iloc[:,0:(2*numWiFiIds)].values    
    """
    
    X = data.iloc[:,0:-4].values    
    y = data.iloc[:,-4:-1].values
    groups = data.iloc[:,-1].values
    del data
    gc.collect()
    return X,y, groups

In [3]:
def plotTrainingResults(resultsDf):
    # subplot to plot
    fig = make_subplots(rows=1, cols=1)
    colors = [ ('#d32f2f', '#ef5350'), ('#303f9f', '#5c6bc0'), ('#00796b', '#26a69a'),
                ('#fbc02d', '#ffeb3b'), ('#5d4037', '#8d6e63')]

    # find number of folds input df
    numberOfFolds = resultsDf['fold'].nunique()
    
    # iterate through folds and plot
    for i in range(numberOfFolds):
        data = df[df['fold'] == i]
        fig.add_trace(go.Scatter(x=data['epoch'].values, y=data['trainPosLoss'].values,
                                mode='lines', visible='legendonly' if i > 0 else True,
                                line=dict(color=colors[i][0], width=2),
                                name='trainPossLoss -Fold{}'.format(i)),row=1, col=1)

        fig.add_trace(go.Scatter(x=data['epoch'], y=data['valPosLoss'].values,
                                 mode='lines+markers', visible='legendonly' if i > 0 else True,
                                 line=dict(color=colors[i][1], width=2),
                                 name='valPosLoss -Fold{}'.format(i)),row=1, col=1)
    fig.show()

## Dataset class

In [11]:
class wiFiFeaturesDataset(Dataset):
    def __init__(self, X_data, y_data, transform=None):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        x = torch.from_numpy(self.X_data[index].astype(np.float32))
        y = torch.from_numpy(self.y_data[index].astype(np.float32))
        return x,y
        #return self.X_data[index], self.y_data[index]
    
    def __len__ (self):
        return len(self.X_data)

## MLP Model class

In [12]:
class wiFiFeaturesMLPModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.lin1 = nn.Linear(in_features=n_input, out_features=512)
        self.lin2 = nn.Linear(in_features=512,     out_features=32)
        self.lin3 = nn.Linear(in_features=32,      out_features=n_output)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(32)
        self.drops = nn.Dropout(0.3)        

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn1(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = self.lin3(x)
        return x

## Compute Device as CPU or GPU

In [13]:
## Device as cpu or tpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


## Preprocessing classes

In [14]:
# for cv
folds = GroupKFold(n_splits=CFG.N_FOLDS)

# for normalizing input data
stdScaler = StandardScaler()

# scaler to handle AMP
scaler = GradScaler()   

## Lr range finder

In [15]:
def plot_lr_finder_results(lr_finder): 
    # Create subplot grid
    fig = make_subplots(rows=1, cols=2)
    # layout ={'title': 'Lr_finder_result'}
    
    # Create a line (trace) for the lr vs loss, gradient of loss
    trace0 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['smooth_loss'],name='log_lr vs smooth_loss')
    trace1 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['grad_loss'],name='log_lr vs loss gradient')

    # Add subplot trace & assign to each grid
    fig.add_trace(trace0, row=1, col=1);
    fig.add_trace(trace1, row=1, col=2);
    iplot(fig, show_link=False)
    #fig.write_html(CFG.MODEL_NAME + '_lr_find.html');

In [16]:
def find_lr(model, optimizer, data_loader, init_value = 1e-8, final_value=100.0, beta = 0.98, num_batches = 200):
    assert(num_batches > 0)
    mult = (final_value / init_value) ** (1/num_batches)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    batch_num = 0
    avg_loss = 0.0
    best_loss = 0.0
    smooth_losses = []
    raw_losses = []
    log_lrs = []
    dataloader_it = iter(data_loader)
    progress_bar = tqdm(range(num_batches))                
        
    for idx in progress_bar:
        batch_num += 1
        try:
            inputs, targets = next(dataloader_it)
            #print(images.shape)
        except:
            dataloader_it = iter(data_loader)
            inputs, targets = next(dataloader_it)

        # Move input and label tensors to the default device
        inputs = inputs.to(device)
        targets = targets.to(device)

        # handle exception in criterion
        try:
            # Forward pass
            y_preds = model(inputs)
            posLoss, floorLoss = criterion(y_preds, targets)
            loss = posLoss + floorLoss
        except:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results 
                    
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) *loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 50 * best_loss:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results
        
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
        
        #Store the values
        raw_losses.append(loss.item())
        smooth_losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print info
        progress_bar.set_description(f"loss:{loss.item()},smoothLoss: {smoothed_loss},lr:{lr}")

        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
    
    grad_loss = np.gradient(smooth_losses)
    lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                         'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
    return lr_finder_results

In [17]:
if CFG.LR_FIND == True:
    # create dataset instance
    tempX, tempY,_ = getBuildingData(buildingCsvPath=buildingsList[0])
    tempX = stdScaler.fit_transform(tempX)
    tempTrainDataset = wiFiFeaturesDataset(tempX, tempY)
    tempTrainDataloader = DataLoader(tempTrainDataset, batch_size= CFG.TRAIN_BATCH_SIZE, shuffle=True,
                          num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
    
    # create model instance   
    model = wiFiFeaturesMLPModel(n_input=tempX.shape[1], n_output=3)
    model.to(device);
    
    # optimizer function, lr schedulers and loss function
    optimizer = getOptimizer(model)
    criterion = competitionMetric
    lrFinderResults = find_lr(model, optimizer, tempTrainDataloader)
    plot_lr_finder_results(lrFinderResults)
    del tempX, tempY, tempTrainDataset, tempTrainDataloader, model, optimizer, criterion

## Train & Validate one section

In [18]:
def trainValidateOneFold(i_fold, model, optimizer, scheduler, dataloader_train, dataloader_valid):
    trainFoldResults = []
    bestValScore = np.inf
    bestEpoch = 0

    for epoch in range(CFG.N_EPOCHS):
        #print('Epoch {}/{}'.format(epoch + 1, CFG.N_EPOCHS))
        model.train()
        trainPosLoss = 0.0
        trainFloorLoss = 0.0

        # training iterator
        tr_iterator = iter(dataloader_train)

        for idx in range(len(dataloader_train)):
            try:
                inputs, targets = next(tr_iterator)
            except StopIteration:
                tr_iterator = iter(dataloader_train)
                inputs, targets = next(tr_iterator)

            inputs = inputs.to(device)
            targets = targets.to(device)  

            # builtin package to handle automatic mixed precision
            with autocast():
                # Forward pass
                y_preds = model(inputs)   
                posLoss, floorLoss = criterion(y_preds, targets)
                loss = posLoss # + floorLoss

                # Backward pass
                scaler.scale(loss).backward()        
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad() 

                # log the necessary losses
                trainPosLoss   += posLoss.item()
                trainFloorLoss += floorLoss.item()

                if scheduler is not None: 
                    if CFG.SCHEDULER == 'CosineAnnealingWarmRestarts':
                        scheduler.step(epoch + idx / len(dataloader_train)) 
                    # onecyle lr scheduler / CosineAnnealingLR scheduler
                    else:
                        scheduler.step()
                    
        # Validate
        model.eval()
        valTotalLoss = 0.0
        valPosLoss = 0.0
        valFloorLoss = 0.0
        val_preds = []
        val_targets = []
        valid_iterator = iter(dataloader_valid)

        for idx in range(len(dataloader_valid)):
            try:
                inputs, targets = next(valid_iterator)
            except StopIteration:
                valid_iterator = iter(dataloader_valid)
                inputs, targets = next(valid_iterator)

            inputs = inputs.to(device)
            targets = targets.to(device)  

            # forward prediction
            with torch.no_grad():    
                y_preds = model(inputs)

            # store predictions and targets to compute metrics later
            val_targets.append(targets)
            val_preds.append(y_preds)

        val_preds = torch.cat(val_preds, 0)
        val_targets = torch.cat(val_targets, 0)
        valPosLoss, valFloorLoss = competitionMetric(val_preds, val_targets)
        valScore = valPosLoss #+ valFloorLoss
        
        # store results
        trainFoldResults.append({ 'fold': i_fold, 
                                  'epoch': epoch, 
                                  'trainPosLoss': trainPosLoss / len(dataloader_train), 
                                  'trainFloorLoss': trainFloorLoss / len(dataloader_train), 
                                  'valPosLoss': valPosLoss.item() , 'valFloorLoss': valFloorLoss.item()})

        # print to console
        #if CFG.PRINT_N_EPOCH:
            #print(f"Fold :{i_fold},Epoch:{epoch},trainLoss={trainPosLoss/len(dataloader_train):.4f},{trainFloorLoss/len(dataloader_train) :.4f}, valLoss={valPosLoss.item():.4f},{valFloorLoss.item():.4f}")
        
        # save best models        
        if(valScore < bestValScore):
            # reset variables
            bestValScore = valScore
            bestEpoch = epoch

            # save model weights
            torch.save({'model': model.state_dict(), 'val_preds':val_preds, 'val_targets':val_targets}, 
                        f"{modelOutputDir}/{CFG.MODEL_NAME}_fold_{i_fold}_epoch{epoch}.pth")

    print(f"For Fold {i_fold}, Best validation score of {bestValScore} was got at epoch {bestEpoch}")                
    return trainFoldResults

## Training & Validation main function

In [19]:
%%time
if CFG.TRAIN == True:
    # placeholder to store results
    trainResults = []

    # for building in buildingList:
    X, y, groups = getBuildingData(buildingCsvPath=buildingsList[0])
    print(X.shape, y.shape, groups.shape)

    for i_fold, (train_idx, valid_idx) in enumerate(folds.split(X=X, y=y[:,0],groups=groups)):
        if i_fold in CFG.FOLD_TO_TRAIN:
            print("Fold {}/{}".format(i_fold + 1, CFG.N_FOLDS))
            # print(train_idx.shape, valid_idx.shape)
            
            # splitting into train and validataion sets
            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
                                        
            # normalize input
            #print(f"Before stdscaler : train_mean{X_train.mean(), X_train.std(), X_valid.mean(), X_valid.std()}")
            X_train = stdScaler.fit_transform(X_train)
            X_valid = stdScaler.transform(X_valid)
            #print(f"After stdscaler : train_mean{X_train.mean(), X_train.std(), X_valid.mean(), X_valid.std()}")
            #print(f"x,y shapes = {X_train.shape, y_train.shape, X_valid.shape, y_valid.shape}")
            
            # move to GPU if present
            #X_train = torch.from_numpy(X_train.astype(np.float32)).to(device)
            #y_train = torch.from_numpy(y_train.astype(np.float32)).to(device)
            #X_valid = torch.from_numpy(X_valid.astype(np.float32)).to(device)
            #y_valid = torch.from_numpy(y_valid.astype(np.float32)).to(device)
            
            # create torch Datasets and Dataloader for each fold's train and validation data
            dataset_train = wiFiFeaturesDataset(X_train, y_train)
            dataset_valid = wiFiFeaturesDataset(X_valid, y_valid)            
            dataloader_train = DataLoader(dataset_train, batch_size= CFG.TRAIN_BATCH_SIZE, shuffle=True,
                                      num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
            dataloader_valid = DataLoader(dataset_valid, batch_size= CFG.TEST_BATCH_SIZE, shuffle=True,
                                      num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
            
            # supervised model instance and move to compute device
            model = wiFiFeaturesMLPModel(n_input=X_train.shape[1], n_output=3)
            model.to(device);
            # print(f"there are {find_no_of_trainable_params(model)} params in model")
            
            # optimizer function, lr schedulers and loss function
            optimizer = getOptimizer(model)
            scheduler = getScheduler(optimizer, dataloader_train)
            criterion = competitionMetric
            # print(f"optimizer={optimizer}, scheduler={scheduler}, loss_fn={criterion}")

            # train and validate single fold
            foldResults = trainValidateOneFold(i_fold, model, optimizer, scheduler,\
                                               dataloader_train, dataloader_valid)
            trainResults = trainResults + foldResults

(9296, 10170) (9296, 3) (9296,)
Fold 1/5
For Fold 0, Best validation score of 9.048604965209961 was got at epoch 26
Fold 2/5
For Fold 1, Best validation score of 8.98095417022705 was got at epoch 5
Fold 3/5
For Fold 2, Best validation score of 8.602737426757812 was got at epoch 27
Fold 4/5
For Fold 3, Best validation score of 8.7635498046875 was got at epoch 28
Fold 5/5
For Fold 4, Best validation score of 10.40999698638916 was got at epoch 25
CPU times: user 1h 19min 10s, sys: 21min 42s, total: 1h 40min 53s
Wall time: 1h 42min 7s


In [20]:
trainResults = pd.DataFrame(trainResults)
trainResults['valTotalLoss'] = trainResults['valPosLoss'] + trainResults['valFloorLoss']
trainResults['trainTotalLoss'] = trainResults['trainPosLoss'] + trainResults['trainFloorLoss']
trainResults.head(3)

,fold,epoch,trainPosLoss,trainFloorLoss,valPosLoss,valFloorLoss
0,0,0,49.952648,21.110264,12.569757,22.786444
1,0,1,23.899113,21.098015,14.258412,22.786444
2,0,2,23.148408,21.095223,11.834853,22.786444


In [ ]:
plotTrainingResults(trainResults)

In [22]:
bestResults = []
for fold in range(CFG.N_FOLDS):
    foldDf = trainResults[trainResults['fold']== fold]
    bestResults.append(foldDf.iloc[np.argmin(foldDf['valTotalLoss'].values),:])
bestResults =pd.DataFrame(bestResults)
bestResults

,fold,epoch,trainPosLoss,trainFloorLoss,valPosLoss,valFloorLoss,valTotalLoss,trainTotalLoss
26,0.0,26.0,21.343768,21.096619,9.048605,22.786444,31.835049,42.440387
55,1.0,5.0,22.121548,22.075527,8.980954,18.889187,27.870141,44.197075
127,2.0,27.0,21.229462,21.943548,8.602737,19.427420,28.030157,43.173010
178,3.0,28.0,21.618944,21.689361,8.763550,20.414200,29.177750,43.308305
225,4.0,25.0,20.966408,20.377481,10.409997,25.658955,36.068952,41.343890


In [23]:
# save results to csv
trainResults.to_csv(f"{modelOutputDir}/{getBuildingName(buildingsList[0])}_{CFG.MODEL_NAME}_trainResults.csv")
bestResults.to_csv(f"{modelOutputDir}/{getBuildingName(buildingsList[0])}_{CFG.MODEL_NAME}_bestResults.csv")